In [1]:
import sys
sys.path.insert(0, '/Users/finn/Desktop/Capstone-BSDE/files')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from bsde.config import ConfigHJB1, ConfigLSMC, ConfigDeepBSDE, ConfigFBSNN
from bsde.solver.lsmc import LSMCLinear
from bsde.solver.deep_bsde import DeepBSDESolver
from bsde.dynamics.liquidation1 import HJB_liquidation1_FBSDE, HJB_liquidation1_solver
tf.compat.v1.enable_eager_execution()

2022-12-09 19:52:06.288707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow version 2.11.0
Instructions for updating:
non-resource variables are not supported in the long term


Algo & HF trading - P140

$\newcommand{\a}{\alpha} \newcommand{\s}{\sigma} \newcommand{\half}{\frac{1}{2}} \newcommand{\F}{\mathcal{F}} \newcommand{\P}{\mathbb{P}} \newcommand{\par}{\partial} \newcommand{\R}{\mathbb{R}} \newcommand{\argmin}{\arg\!\min} \newcommand{\E}{\mathbb E} \newcommand{\lb}{\left [} \newcommand{\rb}{\right ]} \newcommand{\U}{\mathcal{U}}$ $\newcommand{\lm}{\begin{pmatrix}} \newcommand{\rm}{\end{pmatrix}} \newcommand{\eps}{\varepsilon}$

The model follows

\begin{split}
dS_t &= -g(\nu_t) dt + \sigma_s dW^t \\
dQ_t & = -\nu_t dt \\
\hat S_t & = S_t - \left(\half \Delta + f(\nu_t) \right)
\end{split}




The value function is, under the assumption that $\Delta = 0$ and $f(v) = k v$, 
$$ H(t,s,q) = \sup_{\nu_t \in \mathcal A(t,T)} E_t\left( \int_t^T \hat S_u \nu_u du \right) = \sup_{\nu_t \in \mathcal A(t,T)} E_t\left( \int_t^T (S_u - k \nu_u) \nu_u du \right) $$

which is solution to:
$$ \partial_t H + \half \sigma_S^2 \partial_{ss}H + \sup_v {(s-kv)v - v\partial_q H} = 0 $$

The supremum is achieved for $v^* = \frac{1}{2k}(s-\partial_q H)$ and yields
$$ \partial_t H + \half \sigma_S^2 \partial_{ss}H + \frac{1}{4k}(s-\partial_q H)^2 = 0 $$

In order to ensure that $Q_T = 0$, the value funtion must satisfy $\lim_{t \uparrow T} H(t,s,q) = -\infty$ for $q \neq 0$ and $H(T,s,0) = 0$.

This leads to 
$$ H(t,s,q) = qs - q^2 \frac{k}{T-t} $$



For numerical consideration purposes, we need to remove the constraints that $\lim_{t \uparrow T} H(t,s,q) = -\infty$ which can be done by setting $H(T,s,q) = qs -\lambda q^2$ for $\lambda >0,(\lambda>>0)$. This similar to penalizing the remaining inventory at maturity. The analytical solution becomes
$$ H_{\lambda}(t,s,q) = qs - q^2 \left(\frac{1}{\lambda} + \frac{1}{k}(T-t)\right)^{-1} $$






Another numerical trick (that needs to be introduced to deal with the singularity of the PDE) is to consider the solution $H_{\lambda, \eps}$ to the PDE
$$ \partial_t H + \half \sigma_S^2 \partial_{ss}H + \half \eps^2 q^2 \partial_{qq}H  + \frac{1}{4k}(s-\partial_q H)^2 = 0 $$

We now want to cast this PDE with it's corresponding BSDE $Y_t = H(t,X_t) = H(t,S_t, Q_t)$ process.

Noting $X_t = (S_t, Q_t)^{T}$ and $D_x H = (\partial_s H, \partial_q H)^T$, we consider the SDEs:
\begin{split}
dS_t &= \sigma_s dW^S_t \\
dQ_t &= \eps Q_t dW^Q_t \\
\end{split}

with $\langle W^S, W^Q \rangle_t = 0$, the volatility matrix is
$$ \s_t= \lm \s_s & 0 \\ 0 & \eps  Q_t\rm $$

So that 
$$ Z_t = \s_t^T D_x H(t,X_t) = \lm \s_s \partial_s H(t,X_t) & 0 \\ 0 & \eps  Q_t \partial_q H(t,X_t)\rm $$

We now see that we need to set
$$ f(s,q, Z) = \frac{1}{4k}(a \cdot X + A \cdot Z)^2 = \frac{1}{4k}(s-\partial_q H)^2 $$

with 
\begin{split}
a &= (1,0)^T \\
A &= \left(0, -\frac{1}{\eps q}\right)^T \\
\end{split}


<div class="alert alert-block alert-info">
<b>Remark:</b> We could have chosen $dQ_t = \eps dW^Q_t$, but the issue is that the term $(s-\partial_qH)^2$ is in $q^2$ and the term in $\partial_{qq}H$ is a non-zero constant that cannot offset $(s-\partial_qH)^2$. Choosing $dQ_t = \eps Q_t dW^Q_t$ instead is useful to get an analytical expression of $H(t,s,q)$.
    
We could still proceed with the guess $H(t,s,q) = a(t) + qs + q^2 h(t)$, and assuming a PDE constant with $dQ_t = \eps dW^Q_t$, we would get
$$ \partial_t H + \half \sigma_s^2 \partial_{ss}H + \half \eps^2 \partial_{qq} H + \frac{1}{4k}(s-\partial_q H)^2 = a'(t) + q^2 h'(t) + \eps^2 h(t) + \frac{1}{k}q^2 h(t)^2  $$
    
Then we need to choose $a(t)$ and $h(t)$ such that $h'(t) + \frac{1}{k}h^2(t) = 0$ and $a'(t)= h(t)$ which is now trivial and yields for $h(T) \neq 0$:
$$ h(t) = \left( \frac{1}{h(T)} - \frac{1}{k}(T-t) \right)^{-1}$$
$$ a(t) = a(T) + \frac{1}{k} \ln \left(\frac{h(T) - \frac{1}{k}(T-t)}{h(T)} \right) $$
    
If $h(T) = 0$, then $a(t) = \frac{1}{k} \ln(T-t)$ leads to a choice that unfortunately always gives $\lim_{t \uparrow T} H(t,s,q) = -\infty$ even when $q=0$.    
</div>


<div class="alert alert-block alert-info">
<b>Remark:</b> A possible extension, more sophisticated but that could be exact is to write
\begin{split}  
 0 &=  \partial_t H + \half \sigma_S^2 \partial_{ss}H + \frac{1}{4k}(s-\partial_q H)^2 \\
&= \partial_t H + \half \sigma_S^2 \partial_{ss}H + \half \eps^2 q^2 \partial_{qq}H + \frac{1}{4k}(s-\partial_q H)^2 \\
& = \partial_t H + \mathcal L H + f(t, s,q, H, D_xH, D_x^2 H)
\end{split}
    
is a fully-non linear PDE with a 2D-BSDE reprensetation
\begin{split}  
 dY_t &=  -f(t,Y_t, X_t, Z_t, \Gamma_t, A_t) + Z^T_t dW_t \\
dZ_t &= A_t dt + \Gamma_t dW_t \\
\end{split}    
 
Setting $Y_t = u(t,X_t)$, assuming enough regularity and applying Ito's formula yields $Z_t = D_x u$, $\Gamma_t = D^2_x u$ and $A_t = \mathcal L u$.    
    
A standard LSMC simulation of such equation is similar to the Markov FBSDE, but with one more layer of regression:
* $Y_T = g(X_T)$
* $Z^T_t$ is still obtained from $ Z^T_t = \frac{d}{dt}\langle Y,W \rangle_t = E_t(dY_t W_t^T) $
* $\Gamma_t = \frac{d}{dt}\langle Z,W \rangle_t = E_t(dZ_t W_t^T)$
* $A_t = E_t(dZ_t / dt)$ is regressed on a basis of functions.
* $Y_{t_n}$ comes from $Y_{t_{n+1}}$ and the regression $E_t(dY_t / dt) = -f(t,Y_t, X_t, Z_t, \Gamma_t, A_t)$  
</div>

In [3]:
# Simulation parameters
T = 0.25
dt = 1 / 252
N = int(T/dt)
d = 2
d1 = 2
d2 = 1
seed = 42

# Model parameters
x0 = np.array([30., 10000.])   # S_0, q_0
epsilon = 0.001
sig_s = 0.5
lb = 1.
k = 0.001

# config of the pde
cfg_HJB1 = ConfigHJB1(sig_s=sig_s, eps=epsilon, lb=lb, k=k, T=T, d=d, d1=d1, d2=d2)

print('Analytical Y0: {}'.format(x0[0]*x0[1] - x0[1]**2 / (1/lb + T/k)))

Analytical Y0: -98406.37450199202


$$ Y_0 = H_{\lambda}(0,s,q) = qs - q^2 \left(\frac{1}{\lambda} + \frac{T}{k}\right)^{-1} $$

In [4]:
# FBSNN -- Maziar Raissi (current method)
if True:
    tf.compat.v1.disable_v2_behavior()

    # Batch size
    batch_size = 8

    # Config of the solver
    cfg_FBSNN = ConfigFBSNN(x0=x0.reshape(1, d), N=N, M=batch_size, dt=dt, seed=seed, layers=[d+1] + 4*[256] + [1])

    # Train
    HJB_FBSNN = HJB_liquidation1_solver(config_dynamic=cfg_HJB1, config_solver=cfg_FBSNN)
    HJB_FBSNN.train(N_Iter=3 * 10 ** 1, learning_rate=7e-3)  # change the N_Iter and learning rate
    HJB_FBSNN.train(N_Iter=4 * 10 ** 1, learning_rate=2e-3)  # change the N_Iter and learning rate
    HJB_FBSNN.train(N_Iter=3 * 10 ** 1, learning_rate=1e-4)  # change the N_Iter and learning rate

2022-12-09 19:52:11.642924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 19:52:23.271876: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


It: 0, Loss: 7.956e+16, Y0: -6.659, Time: 6.13, Learning Rate: 7.000e-03
It: 10, Loss: 7.952e+16, Y0: 10.813, Time: 0.39, Learning Rate: 7.000e-03
It: 20, Loss: 7.950e+16, Y0: -1.898, Time: 0.39, Learning Rate: 7.000e-03
It: 0, Loss: 7.960e+16, Y0: -2.321, Time: 0.04, Learning Rate: 2.000e-03
It: 10, Loss: 7.953e+16, Y0: -8.421, Time: 0.40, Learning Rate: 2.000e-03
It: 20, Loss: 7.949e+16, Y0: -3.348, Time: 0.44, Learning Rate: 2.000e-03
It: 30, Loss: 7.949e+16, Y0: -15.555, Time: 0.41, Learning Rate: 2.000e-03
It: 0, Loss: 7.957e+16, Y0: -5.283, Time: 0.04, Learning Rate: 1.000e-04
It: 10, Loss: 7.946e+16, Y0: -7.375, Time: 0.41, Learning Rate: 1.000e-04
It: 20, Loss: 7.946e+16, Y0: -8.045, Time: 0.41, Learning Rate: 1.000e-04


In [5]:
# Deep BSDE -- founding paper (will not be considered currrently)

if False:
    tf.compat.v1.enable_v2_behavior()

    # Batch size
    batch_size = 64

    # Config of the solver
    cfg_deep_solver = ConfigDeepBSDE(N=N, dt=dt, seed=42, x0=x0,
                                     y_init_range=[-98000, -98100],
                                     n_hiddens=[10+d, 10+d],
                                     lr_values=[2e-3, 1e-3],
                                     lr_boundaries=[6000],  # change this if needed
                                     n_iterations=1000,     # change this if needed
                                     batch_size=batch_size,
                                     valid_size=64,
                                     report_freq=100,
                                     dtype='float64',
                                     verbose=True)

    # Train
    FBSDE = HJB_liquidation1_FBSDE(config=cfg_HJB1, exclude_spot=True)
    deep_solver = DeepBSDESolver(FBSDE, cfg_deep_solver)
    deep_solver.train()

## Ignore the part below -- Just for testing the BSDE implementation in FBSNN

In [6]:
# Need to enable v2 behavior of tensorflow
# Simulation parameters
M = 2 ** 12
T = 0.25
dt = 1 / 252
N = int(T/dt)
d = 2
d1 = 2
d2 = 1
seed = 42

# Model parameters
x0 = np.array([30., 10000.])   # S_0, q_0
epsilon = 0.001
sig_s = 0.5
lb = 1.
k = 0.001


def phi_tf(t, X, Y, Z):
    """
    Generator of the BSDE

    :param t: M x 1
    :param X: M x d
    :param Y: M x 1
    :param Z: M x d
    :return: Generator, M x 1
    """
    q = X[:, 1:2]  # M x 1
    s = X[:, 0:1]  # M x 1

    A = -1 / (eps * q)  # M x 1
    neg_partial_q = A * Z[:, 1:2]  # M x 1

    return -1 / (4 * self.k) * (s + neg_partial_q) ** 2

def g_tf(X):
    """
    Final condition

    :param X: M x d
    :return: Final condition, M x 1
    """
    q = X[:, 1:2]  # M x 1
    s = X[:, 0:1]  # M x 1
    val = q * s - lb * (q ** 2)  # M x 1
    return val

def mu_tf(t, X, Y, Z):
    """
    Drift of the Forward SDE

    :param t: M x 1
    :param X: M x d
    :param Y: M x 1
    :param Z: M x d
    :return: Drift, M x d
    """
    return tf.zeros(shape=X.shape, dtype='float32')

def sigma_tf(t, X, Y):
    """
    Volatility of the Forward SDE

    :param t: M x 1
    :param X: M x d
    :param Y: M x 1
    :return: Vol, M x d x d
    """
    val1 = tf.repeat(tf.constant([sig_s, 0], dtype='float32')[None, :],
                     X.shape[0],
                     axis=0)  # M x 2
    val2 = tf.matmul(X, tf.constant([[0, 0], [0, epsilon*1]], dtype='float32'))  # M x 2

    return tf.stack((val1, val2), axis=1)  # M x 2 x 2

In [7]:
Dt = np.zeros((M, N + 1, 1))  # M x (N+1) x 1
DW = np.zeros((M, N + 1, d))  # M x (N+1) x D

dt = T / N

Dt[:, 1:, :] = dt
DW[:, 1:, :] = np.sqrt(dt) * np.random.normal(size=(M, N, d))

t = np.cumsum(Dt, axis=1)  # M x (N+1) x 1
W = np.cumsum(DW, axis=1)  # M x (N+1) x D

W = tf.convert_to_tensor(W, dtype='float32')
t = tf.convert_to_tensor(t, dtype='float32')

In [8]:
x0 = tf.constant([[20, 10000]], dtype='float32')
X0 = tf.tile(x0, [M, 1])  # M x D
t0 = t[:, 0, :]
W0 = W[:, 0, :]

for n in range(0, N):
    t1 = t[:, n + 1, :]
    W1 = W[:, n + 1, :]
    X1 = X0 + mu_tf(t0, X0, 0, 0) * (t1 - t0) + tf.squeeze(
        tf.matmul(sigma_tf(t0, X0, 0), tf.expand_dims(W1 - W0, -1)), axis=[-1])

    t0 = t1
    W0 = W1
    X0 = X1



In [9]:
print(X0)

Tensor("add_825:0", shape=(4096, 2), dtype=float32)


In [10]:
# mean of X
print(tf.math.reduce_mean(X0, axis=0))

Tensor("Mean:0", shape=(2,), dtype=float32)


In [11]:
# std of X
print(tf.math.reduce_std(X0, axis=0))

Tensor("reduce_std/Sqrt:0", shape=(2,), dtype=float32)


In [12]:
# Final condition
X0[:, 0:1] * X0[:, 1:2] - X0[:, 1:2]**2

<tf.Tensor 'sub_445:0' shape=(4096, 1) dtype=float32>